### Conditional Edge가 포함된 LangGraph

In [38]:
'''
workflow = StateGraph(State)

workflow.add_conditional_edges(
    "source_node",
    condition_function,
    (
        "condition1": "target_node1",
        "condition2": "target_node2",
        "default": END
    )
)
'''
# source_node에서 condition_function을 실행하여 반환된 결과에 따라 
# target_node1, target_node2, END로 이동한다.

# condition_function은 현재 상태를 평가하고 다음에 어떤 노드로 이동할지
# 결정하는 함수이다. State를 받아 문자열 또는 Enum 값을 반환한다. 이 값은 다음 노드를 결정한다.

# condition_function은 다음과 같이 정의된다.

'''
from typing import Literal


def check_user_input(state: State) -> Literal["question", "command", "unknown"]:

    if state["input"].endswith("?"):
        return "question"
    elif state["input"].startswith("/"):
        return "command"
    else:
        return "unknown"

workflow.add_conditional_edges(
    "parse_user_input",
    check_user_input,
    {
        "question": "answer_question",
        "command": "execute_command",
        "unknown": "ask_clarification"
    }
}
'''
# check_user_input 함수는 input이 물음표로 끝나면 "question"을 반환하고, 
# 느낌표로 끝나면 "command"를 반환하고 그 외에는 "unknown"을 반환한다.


'\nfrom typing import Literal\n\n\ndef check_user_input(state: State) -> Literal["question", "command", "unknown"]:\n\n    if state["input"].endswith("?"):\n        return "question"\n    elif state["input"].startswith("/"):\n        return "command"\n    else:\n        return "unknown"\n\nworkflow.add_conditional_edges(\n    "parse_user_input",\n    check_user_input,\n    {\n        "question": "answer_question",\n        "command": "execute_command",\n        "unknown": "ask_clarification"\n    }\n}\n'

In [41]:
from typing import Literal, Optional, TypedDict
from langgraph.graph import StateGraph, START, END


# 사용자 쿼리에서 위치 정보 추출 (in 뒤에 위치 정보가 있다고 가정)
def extract_location(query: str) -> Optional[str]:
    words = query.split()
    
    if "in" in query:
        index = words.index("in")
        if index + 1 < len(words):
            return words[index + 1]
    return None


# 위치 이름이 3글자 미만인 경우 위치 정보가 모호하다고 판단
def is_ambiguous(location: str) -> bool:
    return len(location) < 3

# 주어진 위치에 대한 날씨 정보 제공
def fetch_weather_data(location: str) -> str:

    weather_data = {
        "Seoul": "sunny, 25°C",
        "New York": "cloudy, 18°C",
        "London": "rainy, 12°C"
    }

    return weather_data.get(location, "Weather data not available")

# 모호한 입력에 대해 사용자에게 추가 정보 요청
def generate_location_clarification(location: str) -> str:
    return f"There are no data with {location}. Please provide a more specific location."

# 날씨 정보 응답 생성
def format_weather_response(location: str, forecast: str) -> str:
    return f"The weather in {location} is {forecast}."


In [42]:
# 1. 상태 정의

# 랭그래프의 각 단계에서 사용되는 데이터 구조 정의
class WeatherState(TypedDict):
    query: str
    location: Optional[str]
    forecast: Optional[str]

# query: 사용자의 입력
# location: 추출된 위치 정보
# forecast: 가져온 날씨 정보

# 2. 노드 정의

# parse_query: 사용자 쿼리에서 위치 정보를 추출합니다.
def parse_query(state: WeatherState):
    location = extract_location(state["query"])
    return {"location": location}

# get_forecast: 주어진 위치에 대한 날씨 정보를 가져옵니다.
def get_forecast(state: WeatherState):
    forecast = fetch_weather_data(state["location"])    
    return {"forecast": forecast}

# clarify_location: 위치가 모호할 경우 명확화 요청을 생성합니다.
def clarify_location(state: WeatherState):
    clarification = generate_location_clarification
    return {"query": clarification}

# generate_response: 최종 응답을 생성합니다.
def generate_response(state: WeatherState):
    clarification = generate_location_clarification(state["location"])
    return {"query": clarification}


In [43]:
# 3. 조건부 엣지 함수 정의
def check_location(state: WeatherState) -> Literal["valid", "ambiguous", "invalid"]:
    if not state["location"]:
        return "invalid" # 위치 정보가 없는 경우
    elif is_ambiguous(state["location"]):
        return "ambiguous" # 위치 정보가 모호한 경우
    else:
        return "valid" # 유효한 위치일 경우
    

1. StateGraph 객체를 생성하여 그래프의 기본 구조를 정의한다.
2. add_node로 각 처리 단계(노드)를 그래프에 추가한다.
3. set_entry_point로 그래프의 시작점을 설정한다.
4. add_conditional_edges로 조건부 라우팅을 정의한다.
5. add_edge로 일반적인 순차적 처리 흐름을 정의한다.
6. compile로 정의된 그래프를 실행 가능한 형태로 변환한다.

In [44]:
print()